In [1]:
import os
from langchain.document_loaders import CSVLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_huggingface import HuggingFaceEndpoint,ChatHuggingFace
from langchain.cache import InMemoryCache
from langchain.globals import set_llm_cache
from dotenv import load_dotenv

In [2]:
# Reset caching
set_llm_cache(InMemoryCache())

In [3]:
load_dotenv() # Load your API key from .env
from huggingface_hub import login
hf_token = os.getenv("HUGGINGFACE_API_TOKEN")
login(hf_token)

C:\Users\bdobhalx\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Load CSV — set Issue column as the main content
loader = CSVLoader(
    file_path=os.path.abspath("tickets.csv"),
    source_column="Issue",  # Column with user queries
    metadata_columns=["Resolution"],  # Preserve these as metadata
    encoding="utf-8-sig" # 
)

documents = loader.load()

In [5]:
# # Check loaded documents and metadata
for doc in documents:
    print("📝 Content:", doc.page_content)
    print("📎 Metadata:", doc.metadata)

📝 Content: LogID: 113551
Symptom: Version # (if applicable):34978 IsScrapBin (please indicate for FSA issue): Report/UI: Run solve Item(s)/ItemGroup(s)/Location(s): Problem Description: Run solve having error
Issue: Run solve having error
UpdaterName: Elaine Liu
📎 Metadata: {'source': 'Run solve having error', 'row': 0, 'Resolution': 'HXVaf have nsmusd for 2 conversion groups, which requires around 7 cell pairs in ww31. ww31 current inv is not good enough to support the conversion.'}
📝 Content: LogID: 113565
Symptom: Version # (if applicable): WW25 version IsScrapBin (please indicate for FSA issue): Report/UI: Item(s)/ItemGroup(s)/Location(s): All capacity groups Problem Description: OneMPS does not have RTF for 2025Q1 for all solve groups. The bucket Range is 202424 to 202513
Issue: OneMPS does not have RTF for 2025Q1 for all solve groups. The bucket Range is 202424 to 202513
UpdaterName: Weai Hong Leong
📎 Metadata: {'source': 'OneMPS does not have RTF for 2025Q1 for all solve groups.

In [6]:
# 2. Load embeddings model and create a vector store
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(documents, embeddings)  # Convert dicts to Documents

C:\Users\bdobhalx\AppData\Local\Temp\ipykernel_9644\1976614936.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [ ]:
# Test retrieval with new data
test_query = "Unable to connect to the VPN"
docs = vectorstore.similarity_search(test_query)
print(f"Retrieved documents: {[d.page_content[:50] for d in docs]}") # Display first 50 characters of each retrieved document

for doc in documents:
    print("📝 Content:", doc.page_content)

📝 Content: LogID: 113551
Symptom: Version # (if applicable):34978 IsScrapBin (please indicate for FSA issue): Report/UI: Run solve Item(s)/ItemGroup(s)/Location(s): Problem Description: Run solve having error
Issue: Run solve having error
UpdaterName: Elaine Liu
📝 Content: LogID: 113565
Symptom: Version # (if applicable): WW25 version IsScrapBin (please indicate for FSA issue): Report/UI: Item(s)/ItemGroup(s)/Location(s): All capacity groups Problem Description: OneMPS does not have RTF for 2025Q1 for all solve groups. The bucket Range is 202424 to 202513
Issue: OneMPS does not have RTF for 2025Q1 for all solve groups. The bucket Range is 202424 to 202513
UpdaterName: Weai Hong Leong
📝 Content: LogID: 113567
Symptom: Version # (if applicable):34994 IsScrapBin (please indicate for FSA issue): Report/UI: Item(s)/ItemGroup(s)/Location(s): Problem Description: Error to pull Limiter Chart By VGG report. FVL prd.
Issue: Error to pull Limiter Chart By VGG report. FVL prd.
UpdaterName: Ashish 

In [7]:
# 3. Create Retrieval Chain
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})
def format_docs(documents):
    return "\n\n".join(
        f"Resolution: {doc.metadata['Resolution']}"
        for doc in documents
  )

In [8]:
# 4. Define a prompt template for the LLM that includes context
prompt = ChatPromptTemplate.from_template("""You are an IT support assistant. 
If a relevant historical resolution exists, respond ONLY with the solution in one clear sentence.
If no solution exists, respond with: "Please create a support ticket for this issue."
Historical Reference:
{context}
Current Question: {question}""")

In [19]:
# 5. llm from hugging face
llm = HuggingFaceEndpoint(
    #repo_id="microsoft/Phi-3-mini-4k-instruct", 
    repo_id="microsoft/Phi-4-mini-instruct",
    task = "text-generation",
    huggingfacehub_api_token = hf_token    
)

In [20]:
# 6. Build Runnable Pipeline
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [21]:
# 7. Run Query
question = "I am unable to connect VPN. Plsease help."
raw_response = rag_chain.invoke(question)
raw_response

StopIteration: 

In [57]:
segments = raw_response.split("Current Question: ")
last_segment = segments[-1]
question = last_segment.split("\n\nAssistant:")[0].strip()
response = last_segment.split("\n\nAssistant:")[1].split("\n\nHuman:")[0].strip()
print("Question:", question)
print("Response:", response)

Question: I am unable to connect to the company's VPN. I've tried multiple times, but it keeps failing.
Response: Please create a support ticket for this issue.


In [43]:
def final_response(raw_output):
    """Extracts the last Q/A pair from repetitive raw output."""
    # Split by question/assistant markers
    segments = raw_output.split("Current Question: ")
    
    if len(segments) < 2:
        return "No valid response found."
    
    # Get last Q/A pair
    last_segment = segments[-1]
    question = last_segment.split("\n\nAssistant:")[0].strip()
    response = last_segment.split("\n\nAssistant:")[1].split("\n\nHuman:")[0].strip()
    
    return {
        "User Query": question,
        "Support Response": response
    }

In [44]:
final_response(raw_response)

{'User Query': 'I checked wrong box in the first time on “Check to Publish XTO. May I know whether there is impact for this?',
 'Support Response': 'There is no impact as no data is published for that location and division.'}